<a href="https://colab.research.google.com/github/beggy22/boosting/blob/main/Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

data = pd.read_csv("/content/datos_arbol_decision.csv")

In [ ]:
data.head()

,Unnamed: 0,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,outcome
0,0,6.0,148.0,72,35.0,0,33.6,0.627,50,1
1,1,1.0,85.0,66,29.0,0,26.6,0.351,31,0
2,2,8.0,183.0,64,0.0,0,23.3,0.672,32,1
3,3,1.0,89.0,66,23.0,94,28.1,0.167,21,0
4,4,0.0,137.0,40,35.0,168,43.1,2.288,33,1


In [ ]:
from sklearn.model_selection import train_test_split
X = data.drop("outcome", axis = 1)
y = data["outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Supresión de advertencias
def warn(*args, **kwargs):
    pass

import warnings
warnings.warn = warn


In [ ]:
# Modelo de Árbol de Decisión
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

# Predicciones
dt_pred = dt_model.predict(X_test)

# Evaluación
print("Decision Tree Classifier:")
print(classification_report(y_test, dt_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, dt_pred))


Decision Tree Classifier:
              precision    recall  f1-score   support

           0       0.82      0.76      0.79        99
           1       0.61      0.69      0.65        55

    accuracy                           0.73       154
   macro avg       0.71      0.72      0.72       154
weighted avg       0.74      0.73      0.74       154

Confusion Matrix:
 [[75 24]
 [17 38]]


In [ ]:
# Modelo de Bosque Aleatorio
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predicciones
rf_pred = rf_model.predict(X_test)

# Evaluación
print("Random Forest Classifier:")
print(classification_report(y_test, rf_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, rf_pred))


Random Forest Classifier:
              precision    recall  f1-score   support

           0       0.81      0.80      0.80        99
           1       0.64      0.65      0.65        55

    accuracy                           0.75       154
   macro avg       0.72      0.73      0.73       154
weighted avg       0.75      0.75      0.75       154

Confusion Matrix:
 [[79 20]
 [19 36]]


In [ ]:
# Modelo de Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

# Predicciones
gb_pred = gb_model.predict(X_test)

# Evaluación
print("Gradient Boosting Classifier:")
print(classification_report(y_test, gb_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, gb_pred))


Gradient Boosting Classifier:
              precision    recall  f1-score   support

           0       0.83      0.80      0.81        99
           1       0.66      0.71      0.68        55

    accuracy                           0.77       154
   macro avg       0.75      0.75      0.75       154
weighted avg       0.77      0.77      0.77       154

Confusion Matrix:
 [[79 20]
 [16 39]]


In [ ]:
# Modelo de XGBoost
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'subsample': [0.6, 0.8, 1.0]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor estimador
best_xgb = grid_search.best_estimator_

# Predicciones
xgb_pred = best_xgb.predict(X_test)

# Evaluación
print("XGBoost Classifier:")
print(classification_report(y_test, xgb_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, xgb_pred))
print(f"Best hyperparameters: {grid_search.best_params_}")


XGBoost Classifier:
              precision    recall  f1-score   support

           0       0.80      0.81      0.80        99
           1       0.65      0.64      0.64        55

    accuracy                           0.75       154
   macro avg       0.72      0.72      0.72       154
weighted avg       0.75      0.75      0.75       154

Confusion Matrix:
 [[80 19]
 [20 35]]
Best hyperparameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}


Análisis Comparativo :
Utilizando otras metricas no utilizadas en los anteriores analisis, he elegido precisión, recall y f1-score,
 y Boosting parece ser el modelo mas preciso comparando los 3, seguido por Bosques Aleatorios

In [ ]:
from pickle import dump

dump(model, open("boosting_y_comparativa.sav", "wb"))
dump(model, open("boosting_y_comparativa.csv", "wb"))